In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from skimage import io
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import h5py
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
import cv2
import zipfile

In [2]:
!pip install ../input/panda-models/Keras_Applications-1.0.8/Keras_Applications-1.0.8 -f ./ --no-index

Looking in links: ./
Processing /kaggle/input/panda-models/Keras_Applications-1.0.8/Keras_Applications-1.0.8
  Created wheel for Keras-Applications: filename=Keras_Applications-1.0.8-py3-none-any.whl size=50704 sha256=e9c0217b86b05c4c203e26e7740fa7d6d043250cf8c037b953ca1e14593d4fe3
  Stored in directory: /root/.cache/pip/wheels/78/37/20/f4bdc07249590d65a2643f044f1795c52671f0cebf6012057b
Successfully built Keras-Applications


In [3]:
!pip install ../input/panda-models/efficientnet-1.1.0/efficientnet-1.1.0 -f ./ --no-index

Looking in links: ./
Processing /kaggle/input/panda-models/efficientnet-1.1.0/efficientnet-1.1.0
  Created wheel for efficientnet: filename=efficientnet-1.1.0-py3-none-any.whl size=14141 sha256=070ad5ae9b5fc4ff700721561955b96008be7a0d1d92eb3af4a4259ccf7c012d
  Stored in directory: /root/.cache/pip/wheels/74/53/6b/c7c5f7f1a0c39debf15a3754d43761919e2524919d66720197
Successfully built efficientnet


In [4]:
import tensorflow as tf
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import BatchNormalization, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras import applications
from efficientnet.keras import EfficientNetB4
from keras import callbacks
from keras.models import Sequential
import tensorflow_addons as tfa
from sklearn.metrics import confusion_matrix
import keras.backend as K

Using TensorFlow backend.


In [5]:
sz = 256
N = 25
N_row = 5
x_new = sz*N_row
y_new = sz*N_row
input_sh = (x_new,y_new,3)
classes = 6
print(input_sh)

(1280, 1280, 3)


In [6]:
pre_test_path = '../input/panda-models/sub_test/sub_test'
test_path = '../input/prostate-cancer-grade-assessment/test_images'
OUT_TRAIN = 'medium_25_256_256.zip'
dir_p = './'

In [7]:
def tile(img):
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    
    #mask = np.pad(mask,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
    #            constant_values=0)
    
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    
    #mask = mask.reshape(mask.shape[0]//sz,sz,mask.shape[1]//sz,sz,3)
    #mask = mask.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    
    if len(img) < N:
        #mask = np.pad(mask,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=0)
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    
    #mask = mask[idxs]
    for i in range(len(img)):
        result.append({'img':img[i], 'idx':i})
    return result

In [8]:
def get_test(img, name):
    
    img_tiles = tile(img)
    
    with zipfile.ZipFile(OUT_TRAIN, 'w') as img_out:
        for t in img_tiles:
            img,idx = t['img'],t['idx']
            img = cv2.imencode('.png',cv2.cvtColor(img, cv2.COLOR_RGB2BGR))[1]
            img_out.writestr(f'{name}_{idx}.png', img)

    file_name = dir_p+OUT_TRAIN
    
    with zipfile.ZipFile(file_name, 'r') as zip: 
        zip.extractall()

    comb_img = np.zeros(shape=(N,sz,sz,3), dtype='int16')
    for i in range(N):
        path = dir_p +name+'_'+str(i)+'.png'
        img_temp = io.imread(path)
        comb_img[i] = img_temp

    comb_img_s = np.copy(comb_img)
    np.random.shuffle(comb_img_s)

    comb_img = comb_img.reshape(N_row, N_row, sz,sz,3)
    comb_img = comb_img.transpose(0,2,1,3,4)
    comb_img = comb_img.reshape(x_new,y_new, 3)
    comb_img = (comb_img/255).astype('float32')

    comb_img_s = comb_img_s.reshape(N_row, N_row, sz,sz,3)
    comb_img_s = comb_img_s.transpose(0,2,1,3,4)
    comb_img_s = comb_img_s.reshape(x_new,y_new, 3)
    comb_img_s = (comb_img_s/255).astype('float32')

    os.remove(file_name)
    for i in range(N):
        name_t = dir_p+name+'_'+str(i)+'.png'
        os.remove(name_t)
    
    return comb_img, comb_img_s

In [9]:
#img_t = io.MultiImage('../input/panda-models/sub_test/sub_test/003046e27c8ead3e3db155780dc5498e.tiff')[1]
#name = '003046e27c8ead3e3db155780dc5498e'
#temp_img, temp_img_s = get_test(img_t, name)

In [10]:
#img_12 = io.imread('../input/panda-models/sub_test/sub_test/003046e27c8ead3e3db155780dc5498e.tiff')
#plt.imshow(img_12, aspect='auto')
#plt.show()

In [11]:
#plt.imshow(temp_img, aspect='auto')
#plt.show()

In [12]:
#plt.imshow(temp_img_s, aspect='auto')
#plt.show()

In [13]:
model2 = tf.keras.models.load_model('../input/panda-models/model_en0_med_256_25_tob_ep_20.h5')

In [14]:
file_path =[]
file_name = []
sub = pd.DataFrame(columns=['image_id','isup_grade'])
if os.path.exists(test_path):
    for dirname, _, filenames in os.walk(test_path):
        for filename in filenames:
            file_path.append(os.path.join(dirname, filename))
            file_name.append(filename)
    for i, fp in enumerate(file_path):
        img = io.MultiImage(fp)[1]
        name = file_name[i].split('.')[0]
        img_arr, img_arr_s = get_test(img, name)
        img_arr = img_arr.reshape(1, x_new,y_new, 3)
        img_arr_s = img_arr_s.reshape(1, x_new,y_new, 3)
        y = model2.predict(img_arr)
        y_p = np.argmax(y, axis=1)
        y_p_int = y_p[0]
        id_tst = file_name[i].split('.')[0]
        pred = [id_tst, y_p_int]
        sub.loc[len(sub)] = pred
else:
    for dirname, _, filenames in os.walk(pre_test_path):
        for filename in filenames:
            file_path.append(os.path.join(dirname, filename))
            file_name.append(filename)
    for i, fp in enumerate(file_path):
        img = io.MultiImage(fp)[1]
        name = file_name[i].split('.')[0]
        img_arr, img_arr_s = get_test(img, name)
        img_arr = img_arr.reshape(1, x_new,y_new, 3)
        img_arr_s = img_arr_s.reshape(1, x_new,y_new, 3)
        y = model2.predict(img_arr)
        y_p = np.argmax(y, axis=1)
        y_p_int = y_p[0]
        id_tst = file_name[i].split('.')[0]
        pred = [id_tst, y_p_int]
        sub.loc[len(sub)] = pred

In [15]:
sub.to_csv('submission.csv', index=False)